In [1]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage
from dotenv import dotenv_values
from langgraph.checkpoint.memory import MemorySaver
import json

memory = MemorySaver()

In [77]:
config = dotenv_values(".env")

In [78]:
class AgentState(TypedDict):
    task: str
    plan: str
    draft: str
    critique: str
    content: List[str]
    revision_number: int
    max_revisions: int

In [79]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=config["OPEN_AI_KEY"])

In [80]:
PLAN_PROMPT = """You are an expert Microsoft PowerPoint writer tasked with writing a high level outline for a successful business presentation. \
Write such an outline for the user provided topic. Give an outline of the PowerPoint along with any relevant notes \
or instructions for the sections."""

In [81]:
RESEARCH_PLAN_PROMPT = """You are a researcher charged with providing information that can \
be used when creating a business PowerPoint presentation. Generate a list of search queries that will gather \
any relevant information. Only generate 3 queries max."""


In [82]:
WRITER_PROMPT = """You are an Business Consultant assistant tasked with developing excellent PowerPoint presentations.\
Generate the best presentation possible for the user's request and the initial outline. \
Create logical sections for the presentation with concise content with an intention to tell a story. \
If the user provides critique, respond with a revised version of your previous attempts. \
Utilize all the information below as needed: 

------

{content}"""

In [83]:
REFLECTION_PROMPT = """You are a partner at a Consulting firm grading a presentation that will be used to review with clients. \
Generate critique and recommendations for the user's submission. \
Provide detailed recommendations, including requests for length, depth, style, etc."""

In [84]:
RESEARCH_CRITIQUE_PROMPT = """You are a researcher charged with providing information that can \
be used when making any requested revisions (as outlined below). \
Generate a list of search queries that will gather any relevant information. Only generate 3 queries max."""


In [85]:
from langchain_core.pydantic_v1 import BaseModel

class Queries(BaseModel):
    queries: List[str]

In [86]:
from tavily import TavilyClient
import os
tavily = TavilyClient(api_key=config["TAVILY_API_KEY"])

In [87]:
def plan_node(state: AgentState):
    messages = [
        SystemMessage(content=PLAN_PROMPT), 
        HumanMessage(content=state['task'])
    ]
    response = model.invoke(messages)
    return {"plan": response.content}

In [88]:
def research_plan_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_PLAN_PROMPT),
        HumanMessage(content=state['task'])
    ])
    # content = state['plan'] or []
    content = []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}

In [89]:
def generation_node(state: AgentState):
    content = "\n\n".join(state['content'] or [])
    user_message = HumanMessage(
        content=f"{state['task']}\n\nHere is my plan:\n\n{state['plan']}")
    messages = [
        SystemMessage(
            content=WRITER_PROMPT.format(content=content)
        ),
        user_message
        ]
    response = model.invoke(messages)
    return {
        "draft": response.content, 
        "revision_number": state.get("revision_number", 1) + 1
    }


In [90]:
def reflection_node(state: AgentState):
    messages = [
        SystemMessage(content=REFLECTION_PROMPT), 
        HumanMessage(content=state['draft'])
    ]
    response = model.invoke(messages)
    return {"critique": response.content}

In [91]:
def research_critique_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_CRITIQUE_PROMPT),
        HumanMessage(content=state['critique'])
    ])
    content = state['content'] or []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}

In [92]:
def powerpoint_node(state: AgentState):
    print("IN POWERPOINT NODE")

In [93]:
def should_continue(state):
    if state["revision_number"] > state["max_revisions"]:
        return "powerpoint"
    return "reflect"

In [94]:
builder = StateGraph(AgentState)

In [95]:
builder.add_node("planner", plan_node)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_node("research_plan", research_plan_node)
builder.add_node("research_critique", research_critique_node)
builder.add_node("powerpoint", powerpoint_node)

In [96]:
builder.set_entry_point("planner")

In [97]:
builder.add_conditional_edges(
    "generate", 
    should_continue, 
    {"powerpoint": "powerpoint", "reflect": "reflect"}
)


In [98]:
builder.add_edge("planner", "research_plan")
builder.add_edge("research_plan", "generate")
builder.add_edge("reflect", "research_critique")
builder.add_edge("research_critique", "generate")
builder.add_edge("powerpoint", END)

In [99]:
graph = builder.compile()

In [100]:
# thread = {"configurable": {"thread_id": "1"}}
# for s in graph.stream({
#     'task': "How can our clients incorporate GenAI into their workflow?",
#     "max_revisions": 2,
#     "revision_number": 1,
# }, thread):
#     print(s)

In [101]:
thread = {"configurable": {"thread_id": "1"}}
events = graph.stream({
    'task': "How can our clients incorporate GenAI into their workflow?",
    "max_revisions": 2,
    "revision_number": 1}, thread)

for event in events:
    print(event)

{'planner': {'plan': '**Title: Incorporating GenAI into Your Workflow**\n\n**I. Introduction**\n- Brief overview of GenAI technology\n- Importance of integrating GenAI into business workflows\n\n**II. Understanding GenAI**\n- Explanation of what GenAI is\n- Benefits of using GenAI in business operations\n- Examples of successful GenAI implementations in various industries\n\n**III. Assessing Your Workflow**\n- Steps to evaluate current workflow processes\n- Identifying areas where GenAI can be integrated for improvement\n- Setting goals for incorporating GenAI into the workflow\n\n**IV. Implementing GenAI**\n- Selecting the right GenAI tools for your business needs\n- Training employees on how to use GenAI effectively\n- Integrating GenAI seamlessly into existing workflows\n\n**V. Best Practices for GenAI Integration**\n- Tips for maximizing the benefits of GenAI in your workflow\n- Overcoming common challenges when implementing GenAI\n- Monitoring and measuring the impact of GenAI on 

/Users/pramaka/Desktop/Projects/LangChain/.venv/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:1377: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(
/Users/pramaka/Desktop/Projects/LangChain/.venv/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:1390: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


{'research_plan': {'content': ['With your data ready, start generative AI integration in your workflow. This process involves creating a clear plan for AI integration in business without disrupting daily tasks. Developing an Integration Plan To ensure a smooth process, follow these steps: Identify integration points in your workflow. Define the role AI will play at each point.', 'SnapLogic provides robust capabilities for integrating workflow automation with Generative AI (GenAI), allowing users to seamlessly build advanced multi-agent conversation systems by combining the GenAI Snap with other Snaps within their pipeline.', 'The benefits of generative AI include faster product development, enhanced customer experience and improved employee productivity, but the specifics depend on the use case. ... GenAI Changes Innovation & Operations ... Clients receive 24/7 access to proven management and technology research, expert advice, benchmarks, diagnostics and more. Fill', 'Improving custom